<a href="https://colab.research.google.com/github/MitchMathieu/cisc452-a1/blob/master/cisc452a1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from tqdm import tqdm

# Get Data

In [2]:
! git clone https://github.com/MitchMathieu/cisc452-a1.git
! ls cisc452-a1

fatal: destination path 'cisc452-a1' already exists and is not an empty directory.
iris_test.txt  iris_train.txt


**Data as an array**

In [0]:
train_inputs = []
train_labels = []

f = open("cisc452-a1/iris_train.txt", 'r')
for line in f:
  line = line.rstrip().split(',')
  train_inputs.append(np.array(line[:-1], dtype=np.float32))
  train_labels.append(line[-1])


test_inputs = []
test_labels = []

f = open("cisc452-a1/iris_test.txt", 'r')
for line in f:
  line = line.rstrip().split(',')
  test_inputs.append(np.array(line[:-1], dtype=np.float32))
  test_labels.append(line[-1])

# Part One: Perceptron from Scratch




In [0]:
class Perceptron(object):

  def __init__(self, num_inputs, epochs, learning_rate):
    self.epochs = epochs
    self.learning_rate = learning_rate
    self.weights = np.zeros((num_inputs + 1, 3))
    # for pocket
    
  def cat_to_vector(self, label):
    # one hot encode the flower type
    if label == "Iris-setosa":
      return np.array([1, 0, 0], dtype=np.float32)
    elif label == "Iris-versicolor":
      return np.array([0, 1, 0], dtype=np.float32)
    elif label == "Iris-virginica":
      return np.array([0, 0, 1], dtype=np.float32)
    
  def vector_to_cat(self, vec):
    if np.array_equal(vec, [1, 0, 0]):
      return "Iris-setosa"
    elif np.array_equal(vec, [0, 1, 0]):
      return "Iris-versicolor"
    elif np.array_equal(vec, [0, 0, 1]):
      return "Iris-virginica"
    
  def get_error(self, label, prediction):
    # convert string label to vector
    label_vec = self.cat_to_vector(label)
    error = []
    
    for i in range(len(label_vec)):
      error.append(label_vec[i]-prediction[i])
      
    return error
    
  def adjust_weights(self, error, inputs):
    # adjust weights based on error
    for i in range(len(error)):
      for j, row in enumerate(self.weights):
        row[i] += self.learning_rate * error[i] * inputs[j]

  def predict(self, inputs):
    # get net vector from dot prod of weights and input vec
    net = np.dot(inputs, self.weights)
    largest_sum = -1
    pred = None
    # three entries in net represent the sums at each of three output percepts  
    for i, sum in enumerate(net):
      if sum > largest_sum:
        largest_sum = sum
        pred = i
    # create a prediction vector
    Y = np.zeros(3)
    Y[pred] = 1
    # Y vec of 3 binary values representing fired/not fired for each percept
    return Y

  def train(self, training_inputs, labels):
    pocket = self.weights
    best_run = 0
    current_run = 0
    for _ in tqdm(range(self.epochs)):
      # create one iterable from inputs and labels
      for inputs, label in zip(training_inputs, labels):
        # add x0 to the input vector
        inputs = np.insert(inputs, 0, 1)
        # predict and adjust weights
        prediction = self.predict(inputs)
        error = self.get_error(label, prediction)
        # check the error and update the pocket
        if np.array_equal(error, [0, 0, 0]):
          current_run += 1
        else:
          if current_run > best_run:
            best_run = current_run
            pocket = self.weights
          current_run = 0
        self.adjust_weights(error, inputs)
        
    # after all training, set the weight equal to what's in the pocket
    self.weights = pocket

  def test(self, test_inputs, labels):
    # open a file to write predictions to
    output_file = open("predictions_perceptron.txt", 'w')
    predictions = []
    
    correct_predictions = 0.0
    # create one iterable from inputs and labels
    for inputs, label in zip(test_inputs, labels):
      # add x0 to the input vector
      inputs = np.insert(inputs, 0, 1)
      # predict and write to file
      prediction = self.predict(inputs)
      # write prediction to output fileand append to list
      output_file.write(f"{str(inputs)},{self.vector_to_cat(prediction)}\n")
      predictions.append(self.vector_to_cat(prediction))
      # compare prediction to true label
      desired_output = self.cat_to_vector(label)

      if np.array_equal(prediction, desired_output):
        correct_predictions += 1.0
    
    output_file.close()
        
    print(f"Number of inputs: {len(test_inputs)}")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {correct_predictions / float(len(labels))}%")
    return predictions

## Training

In [5]:
# instantiate the perceptron class
perceptron = Perceptron(4, 750, 0.5)

#train train train 
perceptron.train(train_inputs, train_labels)

100%|██████████| 750/750 [00:07<00:00, 97.30it/s] 


## Testing

In [6]:
perceptron_predictions = perceptron.test(test_inputs, test_labels)

# need ot capture the data coming back

Number of inputs: 30
Correct predictions: 29.0
Accuracy: 0.9666666666666667%


In [7]:
! cat predictions_perceptron.txt

[1.  5.1 3.5 1.4 0.2],Iris-setosa
[1.  4.9 3.  1.4 0.2],Iris-setosa
[1.  4.7 3.2 1.3 0.2],Iris-setosa
[1.  4.6 3.1 1.5 0.2],Iris-setosa
[1.  5.  3.6 1.4 0.2],Iris-setosa
[1.  5.4 3.9 1.7 0.4],Iris-setosa
[1.  4.6 3.4 1.4 0.3],Iris-setosa
[1.  5.  3.4 1.5 0.2],Iris-setosa
[1.  4.4 2.9 1.4 0.2],Iris-setosa
[1.  4.9 3.1 1.5 0.1],Iris-setosa
[1.  5.  2.  3.5 1. ],Iris-versicolor
[1.  5.9 3.  4.2 1.5],Iris-versicolor
[1.  6.  2.2 4.  1. ],Iris-versicolor
[1.  6.1 2.9 4.7 1.4],Iris-versicolor
[1.  5.6 2.9 3.6 1.3],Iris-versicolor
[1.  6.7 3.1 4.4 1.4],Iris-versicolor
[1.  5.6 3.  4.5 1.5],Iris-versicolor
[1.  5.8 2.7 4.1 1. ],Iris-versicolor
[1.  6.2 2.2 4.5 1.5],Iris-virginica
[1.  5.6 2.5 3.9 1.1],Iris-versicolor
[1.  6.1 3.  4.9 1.8],Iris-virginica
[1.  6.4 2.8 5.6 2.1],Iris-virginica
[1.  7.2 3.  5.8 1.6],Iris-virginica
[1.  7.4 2.8 6.1 1.9],Iris-virginica
[1.  7.9 3.8 6.4 2. ],Iris-virginica
[1.  6.4 2.8 5.6 2.2],Iris-virginica
[1.  6.3 2.8 5.1 1.5],Iris-virginica
[1.  6.1 2.6 5.6 1.4],

**Confusion Matrix**

In [8]:
cm = confusion_matrix(test_labels, 
                      perceptron_predictions, 
                      labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"])

precision = precision_score(test_labels, 
                            perceptron_predictions, 
                            labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"], 
                            average='micro')

recall = recall_score(test_labels, 
                      perceptron_predictions, 
                      labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"], 
                      average='micro')

print(cm)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
Precision: 0.9667
Recall: 0.9667


# Part Two: Using ANN Libraries

In [9]:
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


**Getting Data**

In [0]:
train = pd.read_csv("cisc452-a1/iris_train.txt", header=None)
test = pd.read_csv("cisc452-a1/iris_test.txt", header=None)

x_train = train.iloc[:,0:4]
y_train = train.iloc[:,4]
x_test = test.iloc[:,0:4]
y_test = test.iloc[:,4]

**Convert categorical labels to 3-dimensional vectors**

In [0]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## Training the model

**Make the model**

In [12]:
model = Sequential()

model.add(Dense(10, input_shape=(4,), activation='relu', name='l1'))
model.add(Dense(3, activation='softmax', name='output'))

optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('Neural Network Model Summary: ')
print(model.summary())






Neural Network Model Summary: 
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
l1 (Dense)                   (None, 10)                50        
_________________________________________________________________
output (Dense)               (None, 3)                 33        
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(x_train, y_train, verbose=1, batch_size=5, epochs=200)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/200
120/120 [==============================] - 0s 2ms/step - loss: 1.6998 - acc: 0.3333
Epoch 2/200
120/120 [==============================] - 0s 268us/step - loss: 1.3653 - acc: 0.3417
Epoch 3/200
120/120 [==============================] - 0s 297us/step - loss: 1.1758 - acc: 0.4000
Epoch 4/200
120/120 [==============================] - 0s 242us/step - loss: 1.0554 - acc: 0.3583
Epoch 5/200
120/120 [==============================] - 0s 276us/step - loss: 0.9908 - acc: 0.4000
Epoch 6/200
120/120 [==============================] - 0s 258us/step - loss: 0.9385 - acc: 0.3917
Epoch 7/200
120/120 [==============================] - 0s 257us/step - loss: 0.8964 - acc: 0.4000
Epoch 8/200
120/120 [==============================] - 0s 273us/step - loss: 0.8610 - acc: 0.4250
Epoch 9/200
120/120 [==============================] - 0s 261us/step - loss: 0.8345 - acc: 0.6833
Epoch 10/200
120/120 [===

In [14]:
results = model.evaluate(x_test, y_test)

print(f"Test set loss: {results[0]:.4f}")
print(f"Test set accuracy: {results[1]:.4f}")

tf_predictions = model.predict(x_test)

# TODO 
#  take predictions to categorical
#  output keras predictions to txt file
#  confusion matrices
#  report

30/30 [==============================] - 0s 1ms/step
Test set loss: 0.1484
Test set accuracy: 0.9000


In [0]:
def vec_to_cat(vec):
  if np.array_equal(vec, [1, 0, 0]):
      return "Iris-setosa"
  elif np.array_equal(vec, [0, 1, 0]):
    return "Iris-versicolor"
  elif np.array_equal(vec, [0, 0, 1]):
    return "Iris-virginica"
  
preds_as_strings = []
for pred in tf_predictions:
  biggest_val = -1
  pred_index = -1
  for i, val in enumerate(pred):
    if val > biggest_val:
      biggest_val = val
      pred_index = i
  pred = np.zeros(3)
  pred[pred_index] = 1
  preds_as_strings.append(vec_to_cat(pred))
  

In [0]:
output_file = open("predictions_tf.txt", "w")


for inputs, pred in zip(x_test.values, preds_as_strings):
  string = f"{inputs},{pred}\n"
  output_file.write(string)
  
output_file.close()

In [21]:
! cat predictions_tf.txt

[5.1 3.5 1.4 0.2],Iris-setosa
[4.9 3.  1.4 0.2],Iris-setosa
[4.7 3.2 1.3 0.2],Iris-setosa
[4.6 3.1 1.5 0.2],Iris-setosa
[5.  3.6 1.4 0.2],Iris-setosa
[5.4 3.9 1.7 0.4],Iris-setosa
[4.6 3.4 1.4 0.3],Iris-setosa
[5.  3.4 1.5 0.2],Iris-setosa
[4.4 2.9 1.4 0.2],Iris-setosa
[4.9 3.1 1.5 0.1],Iris-setosa
[5.  2.  3.5 1. ],Iris-versicolor
[5.9 3.  4.2 1.5],Iris-versicolor
[6.  2.2 4.  1. ],Iris-versicolor
[6.1 2.9 4.7 1.4],Iris-versicolor
[5.6 2.9 3.6 1.3],Iris-versicolor
[6.7 3.1 4.4 1.4],Iris-versicolor
[5.6 3.  4.5 1.5],Iris-versicolor
[5.8 2.7 4.1 1. ],Iris-versicolor
[6.2 2.2 4.5 1.5],Iris-virginica
[5.6 2.5 3.9 1.1],Iris-versicolor
[6.1 3.  4.9 1.8],Iris-virginica
[6.4 2.8 5.6 2.1],Iris-virginica
[7.2 3.  5.8 1.6],Iris-versicolor
[7.4 2.8 6.1 1.9],Iris-virginica
[7.9 3.8 6.4 2. ],Iris-virginica
[6.4 2.8 5.6 2.2],Iris-virginica
[6.3 2.8 5.1 1.5],Iris-versicolor
[6.1 2.6 5.6 1.4],Iris-virginica
[7.7 3.  6.1 2.3],Iris-virginica
[6.3 3.4 5.6 2.4],Iris-virginica


In [22]:
cm = confusion_matrix(test_labels, 
                      preds_as_strings, 
                      labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"])

precision = precision_score(test_labels, 
                            preds_as_strings, 
                            labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"], 
                            average='micro')

recall = recall_score(test_labels, 
                      preds_as_strings, 
                      labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"], 
                      average='micro')
print(cm)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

[[10  0  0]
 [ 0  9  1]
 [ 0  2  8]]
Precision: 0.9000
Recall: 0.9000


(uncomment to download text files)

In [0]:
# from google.colab import files
# files.download('predictions_perceptron.txt')
# files.download('predictions_tf.txt') 